In [ ]:
import bluesky
import bluesky.plan_stubs as bps
from apstools.synApps import SscanRecord, SaveData
from ophyd import Device, EpicsSignal, EpicsSignalRO, Component, EpicsMotor
from apstools.plans import run_blocking_function
from ophyd.status import Status
import numpy as np
from epics import caput, caget
import logging
import time
import h5py
import os
import sys
import subprocess
import pathlib
os.environ.pop('QT_QPA_PLATFORM', None)
os.environ.pop('QT_QPA_PLATFORM_PLUGIN_PATH', None)
sys.path.append("/home/beams/STAFF19ID/bluesky/")
# sys.path.append(str(pathlib.Path(__file__).absolute().parent))
from instrument.utils.trajectories import *

In [ ]:
def mkdir(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
            print(f"Created directory: {directory}")
        except OSError as e:
            print(f"Failed to create directory: {directory} - {e}")
    else:
        print(f"Directory already exists: {directory}")
        
def mksubdirs(save_path, subdirs=[]):
    for folder in subdirs:
        path = f"{save_path}{folder}"
        mkdir(path)
        
def pvget(pv):
    try:
        cmd = f'pvget {pv}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e

def pvput(pv, value):
    try:
        cmd = f'pvput {pv} {value}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e

def run_subprocess(command_list):
    try:
        result = subprocess.getstatusoutput(command_list)
    except subprocess.CalledProcessError as e:
        result = e
        pass
    return result 

In [ ]:
class Xspress3(Device):
    ERASE = Component(EpicsSignal, 'det1:ERASE')
    Acquire = Component(EpicsSignal, 'det1:Acquire')
    AcquireTime = Component(EpicsSignal, 'det1:AcquireTime')
    NumImages = Component(EpicsSignal, 'det1:NumImages')
    ArrayCounter_RBV = Component(EpicsSignal, 'det1:ArrayCounter_RBV')
    EraseOnStart = Component(EpicsSignal, 'det1:EraseOnStart')
    DetectorState_RBV = Component(EpicsSignal, 'det1:DetectorState_RBV')
    TriggerMode = Component(EpicsSignal, 'det1:TriggerMode')
    EnableCallbacks = Component(EpicsSignal, 'Pva1:EnableCallbacks')
    Capture = Component(EpicsSignal, 'HDF1:Capture')
    FilePath = Component(EpicsSignal, 'HDF1:FilePath', string=True)
    FileName = Component(EpicsSignal, 'HDF1:FileName', string=True)
    FileNumber = Component(EpicsSignal, 'HDF1:FileNumber')
    FileWriteMode = Component(EpicsSignal, 'HDF1:FileWriteMode')
    FileTemplate = Component(EpicsSignal, 'HDF1:FileTemplate', string=True)
    AutoIncrement = Component(EpicsSignal, 'HDF1:AutoIncrement')
    AutoSave = Component(EpicsSignal, 'HDF1:AutoSave')
    NumCaptured_RBV = Component(EpicsSignal, 'HDF1:NumCaptured_RBV')

    def setup_xspress3(xp3, npts, sample_name, save_path, dwell_time, trigger_mode, reset_counter=False):
        print("in setup_xspress3 function")
        xp3.wait_for_connection()
        # yield from run_blocking_function(pm1.abort) # TODO: re-implement reset function for profile move
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

        if reset_counter: 
            yield from bps.mv(xp3.FileNumber, 0)
        yield from bps.mv(
            xp3.Capture, 0,
            xp3.Acquire, 0,
            )
        yield from bps.mv(
            xp3.ERASE, 1,
            xp3.NumImages, npts,
            xp3.AcquireTime, dwell_time,
            xp3.EraseOnStart, 0,
            xp3.TriggerMode, trigger_mode, 
            xp3.EnableCallbacks, 1, 
            xp3.AutoIncrement, 1,
            xp3.AutoSave, 1,
            xp3.FileWriteMode, 2,
            # xp3.Acquire, 1,
            xp3.FilePath, save_path,
            xp3.FileName, sample_name,
            xp3.FileTemplate, f"%s%s_%05d.h5",
            xp3.Capture, 1,
            )
        
        print("exit in setup_xspress3 function")

In [ ]:
class ProfileMove(Device):
    abort = Component(EpicsSignal, 'Abort')
    num_points = Component(EpicsSignal, 'NumPoints')
    timer_mode = Component(EpicsSignal, 'TimeMode')
    accel = Component(EpicsSignal, 'Acceleration')
    num_pulses = Component(EpicsSignal, 'NumPulses')
    m1_arr = Component(EpicsSignal, 'M1Positions')
    m1_proc = Component(EpicsSignal, 'M1Positions.PROC')
    m1_use = Component(EpicsSignal, 'M1UseAxis')
    m2_arr = Component(EpicsSignal, 'M2Positions')
    m2_proc = Component(EpicsSignal, 'M2Positions.PROC')
    m2_use = Component(EpicsSignal, 'M2UseAxis')
    m3_arr = Component(EpicsSignal, 'M3Positions')
    m3_proc = Component(EpicsSignal, 'M3Positions.PROC')
    m3_use = Component(EpicsSignal, 'M3UseAxis')
    m4_arr = Component(EpicsSignal, 'M4Positions')
    m4_proc = Component(EpicsSignal, 'M4Positions.PROC')
    m4_use = Component(EpicsSignal, 'M4UseAxis')
    m5_arr = Component(EpicsSignal, 'M5Positions')
    m5_proc = Component(EpicsSignal, 'M5Positions.PROC')
    m5_use = Component(EpicsSignal, 'M5UseAxis')
    m6_arr = Component(EpicsSignal, 'M6Positions')
    m6_proc = Component(EpicsSignal, 'M6Positions.PROC')
    m6_use = Component(EpicsSignal, 'M6UseAxis')
    times = Component(EpicsSignal, 'Times')
    fixed_time = Component(EpicsSignal, 'FixedTime')
    build = Component(EpicsSignal, 'Build.PROC')
    exsc = Component(EpicsSignal, 'Execute', kind="omitted")
    readback = Component(EpicsSignalRO, 'Readback')
    exsc_state = Component(EpicsSignal, 'ExecuteState')
    move_mode = Component(EpicsSignal, 'MoveMode')

    def setup_profile_move(pm1, xarr, yarr, dwell_time):
        print("in setup_profile_move function")
        pm1.wait_for_connection()
        # yield from run_blocking_function(pm1.abort) # TODO: re-implement reset function for profile move
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.
        caput(pm1.m1_arr.pvname, list(xarr))
        caput(pm1.m2_arr.pvname, list(yarr))
        yield from bps.mv(
            pm1.m1_use, 1,
            pm1.m2_use, 1,
            pm1.num_points, len(xarr),
            pm1.accel, 0, 
            pm1.timer_mode, 0, 
            pm1.fixed_time, dwell_time,
            # pm1.m1_arr, list(xarr),
            # pm1.m2_arr, list(yarr),
            pm1.m1_proc,1,
            pm1.m2_proc,1,
            pm1.build, 1
            )
        print("exit in setup_profile_move function")



In [ ]:
def setup_scanrecord(scan, xarr, yarr, dwell_time, trig1="", trig2="", trig3="", trig4="", positioner1="", positioner2="", softglue=None):
    print("in setup_scan function")
    scan.wait_for_connection()
    # yield from bps.mv(scaler1.preset_time, ct)  # counting time/point
    yield from run_blocking_function(scan.reset)
    yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

    # scan1.positioners.p1.P1PA not found... using EpicsSignal directly
    caput(scan.P1PA.pvname, list(xarr))
    caput(scan.P2PA.pvname, list(yarr))
    npts = len(x)
    
    caput(tmm.Acquire.pvname, 1) #begin acquiring tetramm 
    caput(xp3.Acquire.pvname, 1) #begin acquiring xspress3
    pvput(postrm.start_.pvname, 1) #begin position stream
    caput(sgz.send_pulses.pvname, 1) #begin sending triggers
    # positioners
    yield from bps.mv(
        scan.positioners.p1.mode, 1,
        scan.positioners.p1.readback_pv, f"{positioner1}.RBV",
        scan.positioners.p1.setpoint_pv, f"{positioner1}",
        scan.positioners.p2.mode, 1,
        scan.positioners.p2.readback_pv, f"{positioner2}.RBV",
        scan.positioners.p2.setpoint_pv, f"{positioner2}",
        scan.positioner_delay, dwell_time,
        scan.number_points, npts
    )
    print("exit in setup_scan function")

In [ ]:
class TetraMM(Device):
    Acquire = Component(EpicsSignal, 'Acquire')
    AcquireMode = Component(EpicsSignal, 'AcquireMode')
    Range = Component(EpicsSignal, 'Range')
    ValuesPerRead = Component(EpicsSignal, 'ValuesPerRead')
    AveragingTime = Component(EpicsSignal, 'AveragingTime')
    FastAveragingTime = Component(EpicsSignal, 'FastAveragingTime')
    FastAverageScan_scan = Component(EpicsSignal, 'FastAverageScan.SCAN')
    EmptyFreeList = Component(EpicsSignal, 'EmptyFreeList')
    TriggerMode = Component(EpicsSignal, 'TriggerMode')
    NumAcquire = Component(EpicsSignal, 'NumAcquire')
    Capture = Component(EpicsSignal, 'netCDF1:Capture')
    FilePath = Component(EpicsSignal, 'netCDF1:FilePath', string=True)
    FileName = Component(EpicsSignal, 'netCDF1:FileName', string=True)
    NumCapture = Component(EpicsSignal, 'netCDF1:NumCapture')
    FileNumber = Component(EpicsSignal, 'netCDF1:FileNumber')
    FileTemplate = Component(EpicsSignal, 'netCDF1:FileTemplate', string=True)
    AutoIncrement = Component(EpicsSignal, 'netCDF1:AutoIncrement')
    AutoSave = Component(EpicsSignal, 'netCDF1:AutoSave')
    WriteFile = Component(EpicsSignal, 'netCDF1:WriteFile')
    FileWriteMode = Component(EpicsSignal, 'netCDF1:FileWriteMode')
    WriteStatus = Component(EpicsSignal, 'netCDF1:WriteStatus')
    FilePathExists_RBV = Component(EpicsSignal, 'netCDF1:FilePathExists_RBV')
    
    def setup_tetramm(tmm, npts, sample_name, save_path, dwell_time, trigger_mode, reset_counter=False):
        print("in setup_tetramm function")
        tmm.wait_for_connection()
        # yield from run_blocking_function(pm1.abort) # TODO: re-implement reset function for profile move
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

        if reset_counter: 
            yield from bps.mv(tmm.FileNumber, 0)
        yield from bps.mv(tmm.Acquire, 0, tmm.Capture, 0)
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.
        yield from bps.mv(
            tmm.TriggerMode, trigger_mode,
            tmm.NumAcquire, npts,
            tmm.NumCapture, npts, 
            tmm.EmptyFreeList, 1,
            tmm.ValuesPerRead, int(100000*dwell_time), 
            tmm.AveragingTime, dwell_time, 
            tmm.FastAveragingTime, dwell_time,
            # tmm.Acquire, 1,
            tmm.AutoIncrement, 1,
            tmm.AutoSave, 1,
            tmm.FileWriteMode, 2,
            tmm.FilePath, save_path,
            tmm.FileName, sample_name,
            tmm.FileTemplate, f"%s%s_%05d.nc",
            tmm.Capture, 1,
            )
        print("exit setup_tetramm function")
        

In [ ]:
class SoftGlueZinq(Device):
    npts = Component(EpicsSignal, 'SG:plsTrn-1_NPULSES')
    period = Component(EpicsSignal, 'SG:plsTrn-1_PERIOD')
    width = Component(EpicsSignal, 'SG:plsTrn-1_WIDTH')
    clr1_proc = Component(EpicsSignal, 'SG:UpDnCntr-1_CLEAR_Signal.PROC')
    clr2_proc = Component(EpicsSignal, 'SG:UpDnCntr-2_CLEAR_Signal.PROC')
    clrcntr_proc = Component(EpicsSignal, 'SG:UpCntr-1_CLEAR_Signal.PROC')
    clr_C = Component(EpicsSignal, '1acquireDma.C')
    clr_F = Component(EpicsSignal, '1acquireDma.F')
    clr_D = Component(EpicsSignal, '1acquireDma.D')
    VALF = Component(EpicsSignal, '1acquireDma.VALF')
    VALG = Component(EpicsSignal, '1acquireDma.VALG')
    VALC = Component(EpicsSignal, '1acquireDma.VALC')
    enbl_dma = Component(EpicsSignal, '1acquireDmaEnable')
    usrclk_enable = Component(EpicsSignal, 'SG:DivByN-3_ENABLE_Signal.PROC')
    rst_buffer = Component(EpicsSignal, 'SG:BUFFER-1_IN_Signal.PROC')
    send_pulses = Component(EpicsSignal, 'SG:plsTrn-1_Inp_Signal.PROC')
    def setup_softgluezinq(sgz, npts, frequency): 
        print("in setup_softgluezinq function")
        clk_f = 20E6 # master clock
        period = int(clk_f/frequency) #period in number of clock cycles 
        width = int(0.000001 * clk_f) #1us in number of clock cycles
        
        caput(sgz.usrclk_enable.pvname, "0") #disable user clock 
        yield from bps.mv(sgz.enbl_dma, 0)   #disable position collection
            
        yield from bps.mv(
            sgz.npts, npts,       #set npts
            sgz.period, period,   #set period in # of clock cycles
            sgz.width, width,     #pulse width in # of clock cycles
            sgz.clr1_proc, 1,     #clear interferometer 1
            sgz.clr2_proc, 1,     #clear interfoerometer 2
            # sgz.clr_C, 1,         #clears SOMETHING
            # sgz.clr_F, 1,         #clears clears cbuff
            sgz.clr_D, 1,         #clears plots
            # sgz.rst_buffer, 1,    #resets buff (do i need this?)
            sgz.enbl_dma, 1,      #enables position collection
            )
        caput(sgz.usrclk_enable.pvname, "0") #enables user clock 
        print("exit setup_softgluezinq function")

In [ ]:
class PositionerStream(Device):
  reset_ = Component(EpicsSignal, 'reset')
  start_ = Component(EpicsSignal, 'start')
  stop_ = Component(EpicsSignal, 'stop')
  status = Component(EpicsSignal, 'status')
  outputFile = Component(EpicsSignal, 'outputFile')

  def setup_positionstream(filename, filepath):
    print("in setup_positionstream function")
    cmd = f"pvput -r \"filePath\" posvr:outputFile \'{{\"filePath\":\"{filepath}\"}}\'"
    print(run_subprocess(cmd)[1])
    cmd = f"pvput -r \"fileName\" posvr:outputFile \'{{\"fileName\":\"{filename}\"}}\'"
    print(run_subprocess(cmd)[1])
    print("exit setup_positionstream function")

In [ ]:
def create_master_file(basedir, sample_name, groups = ["xspress3", "eiger", "mda", "tetramm", "positions"]):
    with h5py.File(f"{basedir}/{sample_name}_master.h5", "w") as f:
        for group in groups:
            f.create_group(group)
            files = os.listdir(f"{basedir}/{group}")
            if group == "tetramm":
                files = [file for file in files if file.split(".")[-1]=="nc"]
                string_data = [s.encode('utf-8') for s in files]
                dset = f[group].create_dataset("fnames", data=string_data)
                # print(string_data)
            elif group == "mda":
                files = [file for file in files if file.split(".")[-1]=="mda"]
                string_data = [s.encode('utf-8') for s in files]
                dset = f[group].create_dataset("fnames", data=string_data)
                # print(string_data)
            elif group == "xspress3": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                for file in files:
                    print(f"./{group}/{file}")
                    f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/entry")
            elif group == "positions": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                for file in files:
                    print(f"./{group}/{file}")
                    f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/stream")

In [ ]:
def run_scan(scan_type="fly", trajectory="snake", loop1="2idsft:m1", loop2="2idsft:m2", dwell_time="10", 
             devices={"flyXRF":"XSP3_1Chan:", "tetramm:":"2idsft:TetrAMM1", "scanrecord":"2idsft:scan1", "softglue": "2idMZ1:", "positions":"posvr:"},
             sample_name="sample_name", pi_directory = "/mnt/micdata1/save_dev/", comments="",
             l1_center=0, l1_size=0.01, l1_width=0.5, l2_center=0, l2_size=0.01, l2_width=0.5,
             ):
        
    print("parsing parameters")
    x, y, t = snake(dwell_time, l1_size, m1_center, l2_center, l1_width, m2_width)
    x = list(np.round(x, 4))
    y = list(np.round(y, 4))
    npts = len(x)
    frequency= 1/dwell_time/1000
    
    folder_name = sample_name.strip("_")
    save_path = f"{pi_directory}{folder_name}/"
    mkdir(save_path)
    devices= [device for device in devices.keys()]
    subdirs = [detector for detector in devices if detector == ("flyXRF" or "tetramm" or "positions" or "eiger" or "mda" or "xmap")]
    mksubdirs(save_path, subdirs)
    
    print("setting up positioners")
    #TODO: of the parameters in loop1-loop4, figure out which are motors somehow or hardcode them in the devices folder and then import them here. 

    """trigger source"""
    use_softglue_triggers = "softglue" in devices
    
    print("setting up devices")
    for device in devices.keys(): 
        if device == "flyXRF":
            if use_softglue_triggers: 
                trigger_mode = 3 #ext trigger
            else: 
                trigger_mode = 1 #internal
            xp3 = Xspress3(devices[device], name="xp3")
            savepath = f"{save_path}{device}"
            yield from Xspress3.setup_xspress3(xp3, npts, sample_name, savepath, trigger_mode, dwell_time, reset_counter=False)
            
        elif device == "tetramm":
            if use_softglue_triggers: 
                trigger_mode = 1 #ext trigger
            else: 
                trigger_mode = 0 #internal
            tmm = TetraMM(devices[device], name="tmm")
            savepath = f"{save_path}tetramm"
            yield from TetraMM.setup_tetramm(tmm, npts, sample_name, savepath, trigger_mode, dwell_time, reset_counter=False)
        
        elif device == "scanrecord":
            trig1 = tmm.Acquire.pvname # tetramm 
            trig2 = xp3.Acquire.pvname #xspress3
            caput(sgz.send_pulses.pvname, 1) #begin sending triggers
            scan = SscanRecord(devices[device], name="scan")
            yield from setup_scanrecord(scan, xarr, yarr, dwell_time, trig1="", trig2="", trig3="", trig4="", positioner1="", positioner2="", softglue=None):

        elif device == "softglue":
            sgz = SoftGlueZinq(devices[device], name = "sgz")
            yield from SoftGlueZinq.setup_softgluezinq(sgz, npts, frequency)
            
        elif device == "positions": 
            if "xspress3" in devices:
                filenumber = "{:05d}".format(xp3.FileNumber-1)
            else: 
                #TODO: figure out a way to increment softglue filenumber whvery time it closes and a way to reset counter. 
                print("file number not tracked. Not sure how else to set file name if not based on another detector's filenumber")
            postrm = PositionerStream(devices[device], name="postrm")
            filename = f"positions_{filenumber}.h5"
            PositionerStream.setup_positionstream(filename, f"{save_path}positions") 
        else: 
            print(f"unknown device: {deviec}")
    
    print("executing scan")
    # st = Status(timeout=20)
    st = Status()
    def watch_execute_scan(old_value, value, **kwargs):
        # Watch for scan1.EXSC to change from 1 to 0 (when the scan ends).
        if old_value == 1 and value == 0:
            # mark as finished (successfully).
            st.set_finished()
            # Remove the subscription.
            scan.execute_scan.clear_sub(watch_execute_scan)

    # TODO need some way to check if devices are ready before proceeding. timeout and exit with a warning if something is missing. 
    # if motors.inpos and pm1.isready and tmm.isready and xp3.isready and sgz.isready and postrm.isready: 
    time.sleep(1)
    pvput(postrm.start_.pvname, 1) #begin position stream
    yield from bps.mv(scan1.execute_scan, 1)
    scan1.execute_scan.subscribe(watch_execute_scan)
    yield from run_blocking_function(st.wait)

    pvput(postrm.stop_.pvname, 1) #stop position stream
    """Set up masterFile"""
    create_master_file(save_path, sample_name, subdirs)
    
    print("end of plan\n")